## solving QP primal

In [ ]:
using Random
using OSQP
using MathOptInterface

const MOI = MathOptInterface;
const MOIU = MathOptInterface.Utilities;

In [ ]:
n = 20 # variable dimension
m = 10 # no of inequality constraints
p = 5; # no of equality constraints

## create a non-trivial QP problem

In [ ]:
x̂ = rand(n)
P = rand(n, n)
P = P'*P # ensure PSD
q = rand(n)
G = rand(m, n)
h = G*x̂ + rand(m)
A = rand(p, n)
b = A*x̂;

In [ ]:
model = OSQP.Optimizer()
x = MOI.add_variables(model, n);

In [ ]:
# define objective

quad_terms = MOI.ScalarQuadraticTerm{Float64}[]
for i in 1:n
    for j in 1:n
        push!(quad_terms, MOI.ScalarQuadraticTerm(P[i,j],x[i],x[j]))
    end
end

objective_function = MOI.ScalarQuadraticFunction(MOI.ScalarAffineTerm.(q, x),quad_terms,0.0)
MOI.set(model, MOI.ObjectiveFunction{MOI.ScalarQuadraticFunction{Float64}}(), objective_function)
MOI.set(model, MOI.ObjectiveSense(), MOI.MIN_SENSE)

In [ ]:
# add constraints
for i in 1:m
    MOI.add_constraint(model,MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(G[i], x), -h[i]),MOI.LessThan(0.))
end

for i in 1:p
    MOI.add_constraint(model,MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(A[i], x), -b[i]),MOI.EqualTo(0.))
end

for i in 1:n
    MOI.add_constraint(model,MOI.SingleVariable(x[i]),MOI.LessThan(1.))
    MOI.add_constraint(model,MOI.SingleVariable(x[i]),MOI.GreaterThan(0.))
end

In [ ]:
MOI.optimize!(model)